# Guía para Trabajo Práctico de  Inter Process Comunication
## Sistemas Operativos



![Logo](static/unpsjb.png)


<div style="text-align: right">
UNPSJB - FI - Trelew
</div>

# Antes de empezar

## Esta guía está versionada con Git

Ésta guía se encuentra en un repositorio Git en [Github](https://github.com/UNPSJB/so-tp4-2015) y se puede descargar con el siguiente comando:

```bash

git clone https://github.com/UNPSJB/so-ipc-2015.git

```

## Versión online
Existe una [versión online](http://nbviewer.ipython.org/format/slides/github/UNPSJB/so-ipc-2015/blob/master/SO_IPC_Guia.ipynb#/), si hubiese cambios, se verían reflejados en ese enlace.


# IPC y visualización con Processing.py

## Objetivos

En este práctico, cada grupo resolverá un problema clásico de concurrencia (**Filsofos Comensales**, **Barbero Dormilón**, **La panadería**, **Fumadores**, etc), y generará una interfase visual que represente en pantalla lo que está sucediendo a cada momento.


### Herramientas

 - Procssing.py
     - Visualización
 - netcat
     - Convertir la salida estandar de un proceso en comunicación con otro proceso mediante TCP
     - Utilizando *pipes* podemos hacer que el **stdout** sea enviado mediante un socket.
 - printf
     - Generar JSON *a mano* con la ayuda de %d (enteros), %f (flotantes), %s (cadenas), etc.


In [167]:
%%file static/esquema.dot
// Esquema graphviz del ejemplo de más abajo
digraph G {
    subgraph cluster_0 {
        // node[shape=square];
        style=filled;
        color=lightgrey;
        prog_a[label="prog_a "];
        prog_b[label="prog_b "];
        prog_c[label="prog_c "];
        prog_d[label="prog_d "];
        label = "procesos de simulacion en C";
     }
    subgraph cluster_1 {
        label="Processing.py"
        th0 [label="Hilo espera clientes/accept()"]
        th1 [label="Hilo 1", color="red"];
        th2 [label="Hilo 2", color="red"];
        th3 [label="Hilo 3", color="red"];
        th4 [label="Hilo 4", color="red"];
        vis [label="Visualización"];
    }
    prog_a -> th1 [label=" netcat"];
    prog_b -> th2 [label=" netcat"];
    prog_c -> th3 [label=" netcat"];
    prog_d -> th4 [label=" netcat"];
    
    th1 -> vis;
    th2 -> vis;
    th3 -> vis;
    th4 -> vis;
    
    th0->th1;
    th0->th2;
    th0->th3;
    th0->th4;    
}

Overwriting static/esquema.dot


In [168]:
from IPython.display import *
!dot -Tpng static/esquema.dot -o static/esquema.png

![esquema](static/esquema.png?fooo)

# Processing

Processing es un entorno de de programación visual.

Processing tiene una interfase muy similar a la de Arduino y suele ser utilizado como complemento de visualización para sistemas embebidos libres.

![](static/processing_example.png?afa)


## Programación básica en Processing

&nbsp;

```java

void setup() {
    // Definir el tamaño de la pantalla
    size(500, 500);
}

void loop() {
    // Dibujar un rectángulo de en la posición x=10, y=10 y con 20 píxeles de ancho y alto
    rect(10, 10, 20, 20);
}
```


Existen muchas funciones para dibujar en pantalla en la [Referencia](https://processing.org/reference/)

### Escuchando a sockets con Processing
Este fragmento permite escuchar a cualquier proceso que envíe datos al puerto **4455**.
En este caso decimos que **Processing** se está comportando como **ervidor**, aunque no reponda nada.
```java
// Example by Tom Igoe

import processing.net.*;

int port = 4455;       
Server myServer;        

void setup()
{
  size(400, 400);
  background(0);
  myServer = new Server(this, port);
}

void draw()
{
  // Obtener el próximo cliente
  Client thisClient = myServer.available();
  // Si el cliente es distinto de null y si escribió algo, mostrarlo
  if (thisClient !=null) {
    String whatClientSaid = thisClient.readString();
    if (whatClientSaid != null) {
      println(thisClient.ip() + "t" + whatClientSaid);
    } 
  } 
}
```
Ejemplo tomado de la [referncia](file:///opt/homebrew-cask/Caskroom/processing/2.2.1/Processing.app/Contents/Java/modes/java/reference/libraries/net/Server_available_.html)

### JSON en Processing

Para cargar JSON en Processing, la referencia muestra el siguente código:
```java
JSONObject json;

void setup() {

  json = new JSONObject();

  json.setInt("id", 0);
  json.setString("species", "Panthera leo");
  json.setString("name", "Lion");
  saveJSONObject(json, "data/new.json");
}
```

### Problema!!!

No podemos cargar JSON desde una cadena, sólo desde un archivo! Por lo tanto vamos a cambiar de modo de Processing. 

# Processing.py

- Desarollado para facilitar la programación mediante el lenguaje Python (originalmente para alumnos de escuela primaria)
- Python es un lenguaje con baterias incluidas (es este caso, vamos a poder cargar JSON desde una cadena con la biblioteca **json**)
- Todas las funciones de Processing Java están disponibles en el modo Python de Processing

![Processing.py](static/processing_py.png)

## Instalación
Desde el desplegable de la esquina superior derecha de la ventana seleccionamos **Add mode...**
![Instalación](static/install-processing-py.png)

# Instalación (cont.)
En la ventana utilizamos el botón **Install** 
![Instalación seleción modo](static/install-processing-2.png)

#Comunicando C con Processing.py
## JSON

JSON es una forma de pasar datos estructurados mediante **cadenas**. Muy utilizado en la Web y en bases de datos no relacionales.

Vamos a utilizar este formato para transportar información entre C y Processing por:
 - ser legible para los humanos
 - fácil de generar, ya que cualquier lenguaje de programación permite imprimir cadenas (aunque hay formas más sofisticadas para generarlo)

Processing.py posee un módulo llamado **json** que nos permite cargar una cadena JSON en tipos del lenguaje (diccionarios, listas, enteros, flotantes, booleanos)

In [174]:
import json
cadena_json = '{"a":1, "b": 2, "color": "azul", "encendido": false}'
type(cadena_json)  # es una cadena
data = json.loads(cadena_json)
print data["a"], type(data["a"]) # Es un entero
print data["encendido"], type(data["encendido"]) # Es un booleano


1 <type 'int'>
False <type 'bool'>


## Ejemplo de JSON


```json
{
    "entero": 1,
    "floatante": 2.2,
    "cadena": "soy cadena",
    "objeto": {
        "entero": 2,
        "booleano": true
    }
}
```

### ¿Cómo generamos JSON con C?

Utilizando [**``printf()``**](http://linux.die.net/man/3/printf) y sus comodines *%s*, *%f*, *%d*, etc.
```c

// imprime_json.c

#include <stdio.h>
int main(void) {

    printf("{\"pid\": %d, \"estado\": \"%s\"}", 
        getpid(), 
        "iniciando"
    );
    
    return 0;
}
```

In [176]:
%%file src/imprime_json.c
#include <stdio.h>
#include <unistd.h>

int main(void) {
    /* JSON son cadenas, usemos printf */
    printf("{\"pid\": %d, \"estado\": \"%s\"}", 
        getpid(), 
        "iniciando"
    );
    return 0;
}

Writing src/imprime_json.c


In [177]:
!gcc src/imprime_json.c -o imprime_json
!./imprime_json | python -m json.tool

{
    "estado": "iniciando",
    "pid": 10870
}


### El JSON saliendo por stdout

```bash

pepe@localhost $ ./imprime_json 
{pid: 1234, "estado": "iniciando"}
```



### Usando el JSON mediante un pipe a un cat de sockets

```bash
pepe@localhost $ ./imprime_json | nc localhost 4455
```



# Recibiendo los datos con Python


```python
import socket
import json

conexion = socket.socket()
conexion.bind(('localhost', 4455)) # Extremo de escucha para nc
conexion.listen(10)                # Backlog de conexiones 
while True:
    cliente, direccion = conexion.accept()  # Espera una conexión de netcat
    while True:
        cadena = cliente.recv(100)
        if not data:
            break
        datos = json.loads(cadena)
        print datos   # Hacer algo útil

```

# Recibiendo datos con processing.py
```python

import socket
from thread import start_new_thread
# Vairable global de conexion
conexion = None

def setup():
    global conexion
    size(500, 500)
    conexion = socket.socket()
    conexion.bind(('localhost', 4455))
    conexion.listen(10)
    start_new_thread(espera_clientes, (conexion, ))

def espera_clientes(conexion):
    while True:
        cliente, direccion = conexion.accept()
        print "Llegó cliente desde %s %d" % direccion
        while True:
            cadena = cliente.recv(100)
            if not cadena:
                print "Se fue el cliente!"
                break
            print "El cliente dijo: %s" % cadena
            
def stop():
    if conexion:
        conexion.close()
    
def draw():
    fill(255, 0, 0)
    rect(10, 10, 20, 20)
```

# Ejemplo 1: Cambiando un color



In [188]:
%%file static/ejemplo.dot
digraph G {

    programa_c [label="echo '{\"r\": 128, \"g\": 128, \"b\": 128}'"];
    pipe[label="|"];
    nc[label="netcat localhost 4455"];
    programa_c -> pipe -> nc -> socket;
    
    socket[shape=oval];

    socket -> hilo_aceptar;
    hilo_aceptar -> espera_clientes [rankdir=LR];
}

Overwriting static/ejemplo.dot


In [189]:
!dot -Tpng static/ejemplo.dot -o static/ejemplo.png

![](static/ejemplo.png?asass)

```python
import socket
from thread import start_new_thread
import json

# Vairable global de conexion
conexion = None

r, g, b = 255, 255, 255 # Blanco

IMAGENES = {
   "cuadrado": loadImage("cuadradito.png"),
   "circulo": loadImage("circulito.png"),
}

cajas = {
    1: {"x": 100, "y": 100, "imagen": "cuadrado"}, 
    2: {"x": 0, "y": 200,   "imagen": "circulo"},
    3: {"x": 80, "y": 200,  "imagen": "circulo"},
}

hilos = []

def setup():
    global conexion
    size(500, 500)
    conexion = socket.socket()
    conexion.bind(('localhost', 4455))
    conexion.listen(10)
    start_new_thread(espera_clientes, (conexion, ))

def atiende_cliente(cliente, direccion):
    print "Llegó cliente desde %s %d" % direccion
    while True:
        cadena = cliente.recv(100)
        if not cadena:
            print "Se fue el cliente!"
            break
        try:
            data = json.loads(cadena)
            if 'numero' not in data:
                print "El cliente no dijo que modificar"
            else:
                n = data["numero"]
                
                for atributo in ("x", "y"):
                    cajas[n][atributo] = data[atributo]
                # Si viene la imagen, existe en las cargadas?
                if "imagen" in data:
                    a_cambiar = data["imagen"]
                    if a_cambiar not in IMAGENES:
                        print "No existe %s" % a_cambiar
                    else:
                        cajas[n]["imagen"] = a_cambiar
                    

        except ValueError:
            print "Porblemas con JSON"
        except KeyError as e:
            print "JSON no tiene algun atributo necesario %s" % e


def espera_clientes(conexion):
    global cajas
    while True:
        cliente, direccion = conexion.accept()
        hilo = start_new_thread(atiende_cliente, (cliente, direccion))
        hilos.append(hilo)

def stop():
    if conexion:
        conexion.close()

def draw():
    background(255, 255, 255) # Limpiar
    for n in (1, 2, 3):
        caja = cajas[n]
        imagen = IMAGENES[caja["imagen"]]
        image(imagen, caja["x"], caja["y"])
```

## Generando datos de entrada

### Usando echo 

```bash

pepe@localhost $ echo '{"r": 100, "g": 124, "b": 23}' | nc localhost 4455
```


### Usando nc en modo interactivo

```bash
nc localhost 4455
```

### Pequeño programa en C
```c
// src/colores.c
#include <stdio.h>
#include <random.h>
#include <stdlib.h>

int main(void){
    srand(getpid());
    printf("{\"r\": %d, \"g\": %d, \"b\": %d}", rand()/255, rand()/255, rand()/255);
    return 0;
}
```

In [265]:
from json import load

def extract_source(lines):
    found = False
    for line in lines:
        if not found:
            if 
    pass


def guardar_y_compilar(name,path='SO_IPC_Guia.ipynb'):
    with open(path) as fp:
        notebook = load(fp)
    celss = notebook['cells']
    source_cells = [c for c in cells if c['cell_type'] == 'markdown']
    print len(source_cells)
    cell, text = None, None
    for cell in source_cells:

        text = ''.join(cell['source'])
        if name in text:
            break
    print cell, text

guardar_y_compilar('colores.c')

31
{u'source': [u'### Peque\xf1o programa en C\n', u'```c\n', u'// src/colores.c\n', u'#include <stdio.h>\n', u'#include <random.h>\n', u'#include <stdlib.h>\n', u'\n', u'int main(void){\n', u'    srand(getpid());\n', u'    printf("{\\"r\\": %d, \\"g\\": %d, \\"b\\": %d}", rand()/255, rand()/255, rand()/255);\n', u'    return 0;\n', u'}\n', u'```'], u'cell_type': u'markdown', u'metadata': {u'slideshow': {u'slide_type': u'fragment'}}} ### Pequeño programa en C
```c
// src/colores.c
#include <stdio.h>
#include <random.h>
#include <stdlib.h>

int main(void){
    srand(getpid());
    printf("{\"r\": %d, \"g\": %d, \"b\": %d}", rand()/255, rand()/255, rand()/255);
    return 0;
}
```


In [250]:
!gcc src/colores.c -o src/colores
!src/colores

{"r": 250, "g": 67, "b": 17}

In [249]:
def video(fname, mimetype):
    """Load the video in the file `fname`, with given mimetype, and display as HTML5 video.
    """
    from IPython.display import HTML
    video_encoded = open(fname, "rb").read().encode("base64")
    video_tag = '<video controls alt="test" width="100%" height="100%" src="data:video/{0};base64,{1}">'.format(mimetype, video_encoded)
    return HTML(data=video_tag)


## Ejemplo grabado

[Visualización externa](static/ej.mp4)

In [195]:
video('static/ej.mp4', 'mp4')

# Ejemplo 2: Utilizando imágenes

Para cargar imágenes en **Processing.py** es importante que busquemos del tipo **PNG** porque tienen transparencia, los archivos **JPG** o **BMP** no tienen transparencia. Podemos usar editor de imágenes para [separar un objeto del fondo](http://docs.gimp.org/es/gimp-tutorial-quickie-separate.html).

## Vamos a utilizar imágnes con transparencia
![Imágenes](static/cuadradito_y_circulito.svg)

Estas se pueden hacer con Gimp para eliminar el fondo o generarlas con Inkscape.

```python

import socket
from thread import start_new_thread
import json

# Vairable global de conexion
conexion = None

r, g, b = 255, 255, 255 # Blanco

IMAGENES = {
   "cuadrado": loadImage("cuadradito.png"),
   "circulo": loadImage("circulito.png"),
}

cajas = {
    1: {"x": 100, "y": 100, "imagen": "cuadrado"}, 
    2: {"x": 0, "y": 200,   "imagen": "circulo"},
    3: {"x": 80, "y": 200,  "imagen": "circulo"},
}

hilos = []

def setup():
    global conexion
    size(500, 500)
    conexion = socket.socket()
    conexion.bind(('localhost', 4455))
    conexion.listen(10)
    start_new_thread(espera_clientes, (conexion, ))

def atiende_cliente(cliente, direccion):
    print "Llegó cliente desde %s %d" % direccion
    while True:
        cadena = cliente.recv(100)
        if not cadena:
            print "Se fue el cliente!"
            break
        try:
            data = json.loads(cadena)
            if 'numero' not in data:
                print "El cliente no dijo que modificar"
            else:
                n = data["numero"]
                
                for atributo in ("x", "y"):
                    cajas[n][atributo] = data[atributo]
                # Si viene la imagen, existe en las cargadas?
                if "imagen" in data:
                    a_cambiar = data["imagen"]
                    if a_cambiar not in IMAGENES:
                        print "No existe %s" % a_cambiar
                    else:
                        cajas[n]["imagen"] = a_cambiar
                    

        except ValueError:
            print "Porblemas con JSON"
        except KeyError as e:
            print "JSON no tiene algun atributo necesario %s" % e


def espera_clientes(conexion):
    global cajas
    while True:
        cliente, direccion = conexion.accept()
        hilo = start_new_thread(atiende_cliente, (cliente, direccion))
        hilos.append(hilo)

def stop():
    if conexion:
        conexion.close()

def draw():
    background(255, 255, 255) # Limpiar
    for n in (1, 2, 3):
        caja = cajas[n]
        imagen = IMAGENES[caja["imagen"]]
        image(imagen, caja["x"], caja["y"])
        
```

In [251]:
%save SO_IPC_Guia.ipynb
!make slides
!open SO_IPC_Guia.slides.html
# Commitear
from datetime import datetime
fecha = datetime.now().strftime('%x %X')
!git add -u
!git commit -m "Update $fecha"
!git push origin master

'' was not found in history, as a file, url, nor in the user namespace.
ipython nbconvert --to slides SO_IPC_Guia.ipynb
[NbConvertApp] Using existing profile dir: u'/Users/nahuel/.ipython/profile_default'
[NbConvertApp] Converting notebook SO_IPC_Guia.ipynb to slides
[NbConvertApp] Support files will be in SO_IPC_Guia_files/
[NbConvertApp] Loaded template slides_reveal.tpl
[NbConvertApp] Writing 1799035 bytes to SO_IPC_Guia.slides.html
[master 6927192] Update 04/26/15 21:29:03
 1 file changed, 162 insertions(+), 80 deletions(-)
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.28 KiB | 0 bytes/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: This repository moved. Please use the new location:
remote:   https://github.com/UNPSJB/so-ipc-2015.git
To git@github.com:UNPSJB/so-tp4-2015.git
   5e5a964..6927192  master -> master


In [235]:
!lsof -i TCP:4455 | awk '{ print $2}' | grep -v -i pid | xargs kill

In [ ]:
def save_file():
    pass